[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/reznikmm/ada-howto/advent-2020?filepath=%2Fhome%2Fjovyan%2Fnb%2F24%2F24.ipynb)

# Day 24

We need some standard packages to read input and keep tiles.

In [1]:
with Ada.Text_IO;
with Ada.Containers.Hashed_Maps;

Each tile has adjanced tiles in six directions:

In [2]:
type Hex_Direction is (e, se, sw, w, nw, ne);

Let's map hex grid to 2D plane using this rules:

In [3]:
DX : constant array (Hex_Direction) of Integer :=
  (e => +2, se => +1, sw => -1,
   w => -2, nw => -1, ne => +1);

DY : constant array (Hex_Direction) of Integer :=
  (e => 0, se => -1, sw => -1,
   w => 0, nw => +1, ne => +1);

So each tile has corresponding position:

In [4]:
type Position is record
   X, Y : Integer;
end record;

function "+" (Pos : Position; Dir : Hex_Direction) return Position is
  ((X => Pos.X + DX (Dir), Y => Pos.Y + DY (Dir)));

Let's keep tile's color in a hash map

In [5]:
use type Ada.Containers.Hash_Type;
   
function Hash (Value : Position) return Ada.Containers.Hash_Type is
   (2021 * Ada.Containers.Hash_Type'Mod (Value.X) +
    Ada.Containers.Hash_Type'Mod (Value.Y));
   
package Color_Maps is new Ada.Containers.Hashed_Maps
  (Key_Type        => Position,
   Element_Type    => Boolean,
   Hash            => Hash,
   Equivalent_Keys => "=");

Black : Color_Maps.Map;
   
function Is_Black (Pos : Position) return Boolean is
  (Black.Contains (Pos) and then Black (Pos));

A recursive function will parse a string and find tile position:

In [6]:
function Find (Current : Position; Line : String) return Position;
--  Parse Line and find target position starting from Current

function Find (Current : Position; Line : String) return Position is
   Length : constant array (Hex_Direction) of Positive :=
     (e | w => 1, others => 2);
      
   Dir : Hex_Direction;
begin
   if Line = "" then
      return Current;
   end if;
      
   case Line (Line'First) is
      when 'e' =>
         Dir := e;
      when 'w' =>
         Dir := w;
      when 's' =>
         case Line (Line'First + 1) is
            when 'e' =>
               Dir := se;
            when 'w' =>
               Dir := sw;
            when others =>
               raise Constraint_Error;
         end case;
      when 'n' =>
         case Line (Line'First + 1) is
            when 'e' =>
               Dir := ne;
            when 'w' =>
               Dir := nw;
            when others =>
               raise Constraint_Error;
         end case;
      when others =>
         raise Constraint_Error;
   end case;
      
   return Find
     (Current + Dir,
      Line (Line'First + Length (Dir) .. Line'Last));
end Find;

Define input file and a count

In [7]:
Input  : Ada.Text_IO.File_Type;
Count  : Natural := 0;

Now we can sole Part I:

In [8]:
Ada.Text_IO.Open (Input, Ada.Text_IO.In_File, "/home/jovyan/md/24/input");
  
while not Ada.Text_IO.End_Of_File (Input) loop
   declare
      Line   : constant String := Ada.Text_IO.Get_Line (Input);
      Target : constant Position := Find ((0, 0), Line);
   begin
      if Is_Black (Target) then
         Black (Target) := False;
         Count := Count - 1;
      else
         Black.Include (Target, True);
         Count := Count + 1;
      end if;
   end;
end loop;
   
Ada.Text_IO.Put_Line (Count'Image);

 438


Define a `Flip` subprogram to find next state at given position. For a black tile it will flip nearby white tiles also, but only if they are not flipped yet.

In [9]:
procedure Flip
  (Black : Color_Maps.Map;
   Next  : in out Color_Maps.Map;
   Pos   : Position);
--  Compute Next state of a tile at given position

procedure Flip
  (Black : Color_Maps.Map;
   Next  : in out Color_Maps.Map;
   Pos   : Position)
is
   Adjacent : Natural := 0;
begin
   for Dir in Hex_Direction loop
      Adjacent := Adjacent + Boolean'Pos (Is_Black (Pos + Dir));
   end loop;
      
   if Is_Black (Pos) then
      Next.Insert (Pos, Adjacent in 1 .. 2);
         
      for Dir in Hex_Direction loop  --  Flip whites around Pos
         declare
            Nearby : Position := Pos + Dir;
         begin
            if not Is_Black (Nearby) and then not Next.Contains (Nearby) then
               Flip (Black, Next, Nearby);
            end if;
         end;
      end loop;
   else
      Next.Insert (Pos, Adjacent = 2);
   end if;
end Flip;

Now do 100 rounds

In [10]:
for J in 1 .. 100 loop
   declare
      Next : Color_Maps.Map;
   begin
      for Pos in Black.Iterate loop
         if Black (Pos) then
            Flip (Black, Next, Color_Maps.Key (Pos));
         end if;
      end loop;
      Black.Assign (Source => Next);
   end;
end loop;

And count black tiles:

In [11]:
Count := 0;
   
for J of Black loop
   Count := Count + Boolean'Pos (J);
end loop;
     
Ada.Text_IO.Put_Line (Count'Image);

 4038


----
[Back to Table of Contents](https://github.com/reznikmm/ada-howto/tree/advent-2020) 